In [1]:
import numpy as np
import itertools
import trimesh
import math
import k3d
from time import sleep

from IPython.display import clear_output
import matplotlib.pyplot as plt
%matplotlib inline

# !conda install -c conda-forge pyembree
# !conda install -c conda-forge igl
# !pip install Cython
# !pip install gym

In [5]:
def compute_metrics(env, agent_func, iter_cnt=10, max_iter=30):
    rewards, final_rewards, novp = [], [], []
    for _ in range(iter_cnt):
        state, action = env.reset()
        episode_reward = 0.0
        for t in range(max_iter):
            action = agent_func(state)
            state, reward, done, info = env.step(action)
            # print("REWARD: ", reward)
            env.render(action, state)
            episode_reward += reward

            if done:
                break

        final_reward = env.final_reward()
        # episode_reward += 1.0 / final_reward
        rewards.append(episode_reward)
        final_rewards.append(final_reward)
        novp.append(t + 1)
    return np.mean(rewards), np.mean(final_rewards), np.mean(novp)

In [6]:
from rl.environment import *

env = Environment(illustrate=False, number_of_view_points=1000, model_path="./data/Dude.obj")
env = CombiningObservationsWrapper(env)
env = StepPenaltyRewardWrapper(env)
env = DepthMapWrapper(env)

### Random

In [7]:
%%time
random_agent_func = lambda s : env.action_space.sample()
compute_metrics(env, random_agent_func, max_iter=20)

unable to load materials from: FinalBaseMesh.mtl
specified material (default)  not loaded!
unable to load materials from: FinalBaseMesh.mtl
specified material (default)  not loaded!
unable to load materials from: FinalBaseMesh.mtl
specified material (default)  not loaded!
unable to load materials from: FinalBaseMesh.mtl
specified material (default)  not loaded!
unable to load materials from: FinalBaseMesh.mtl
specified material (default)  not loaded!
unable to load materials from: FinalBaseMesh.mtl
specified material (default)  not loaded!
unable to load materials from: FinalBaseMesh.mtl
specified material (default)  not loaded!
unable to load materials from: FinalBaseMesh.mtl
specified material (default)  not loaded!
unable to load materials from: FinalBaseMesh.mtl
specified material (default)  not loaded!
unable to load materials from: FinalBaseMesh.mtl
specified material (default)  not loaded!


CPU times: user 35min 23s, sys: 32.9 s, total: 35min 56s
Wall time: 6min 23s


(-6.120172533627703, 0.05615134506556711, 13.2)

### DQN

In [ ]:
import torch
from rl.dqn import *

model = torch.load("./models/dude-100vp-3500.pt")
if USE_CUDA:
    model = model.cuda()

In [ ]:
%%time
agent_func = lambda s : model.act(s, epsilon=0.0)
compute_metrics(env, agent_func)